# Import packages

In [1]:
import pandas as pd
from teradataml import *

from utils import clean_text
from constants import TEST_DATA_PATH
from moduels.db import TeradataDatabase

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\os255022\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Log directory cleaned: c:\internship\AMuRD-Iteration-7\src\logs
Logger initialized. All logs will be saved to: c:\internship\AMuRD-Iteration-7\src\logs\borai_20250811_161150.log


## Connect to database

In [2]:
td_db = TeradataDatabase()
td_db.connect()

## Load data to database

In [ ]:
df = pd.read_csv(TEST_DATA_PATH)
df.head()

,Item_Name,class,Brand,Weight,Number of units,Size of units,Price,T.Price,Pack,Unit
0,Americana Okra zero 400 gm,Vegetables & Fruits,Americana,400gm,1,NaN,NaN,NaN,كيس,gm
1,ليمون اداليا 500 جم,"Tins, Jars & Packets",NaN,500جم,1,NaN,NaN,NaN,علبة,جم
2,صلصه هاينز برطمان خصم عرض,"Tins, Jars & Packets",هاينز,NaN,1,NaN,NaN,NaN,علبة,NaN
3,Dasani water 330ML,Water,Dasani,330مل,1,NaN,NaN,NaN,زجاجة,مل
4,بودرة عصير أناناس من سورس، 900 جم,Soft Drinks & Juices,سورس,900جم,1,NaN,NaN,NaN,عبوة,جم


In [ ]:
df = df[["Item_Name", "Brand"]]
df.rename(columns={"Item_Name": "product_name", "Brand": "brand_name"}, inplace=True)

In [ ]:
df.drop_duplicates(subset=["product_name"], inplace=True)
df.dropna(subset=["product_name"], inplace=True)
df

,product_name,brand_name
0,Americana Okra zero 400 gm,Americana
1,ليمون اداليا 500 جم,NaN
2,صلصه هاينز برطمان خصم عرض,هاينز
3,Dasani water 330ML,Dasani
4,بودرة عصير أناناس من سورس، 900 جم,سورس
...,...,...
4767,كادبورى شوكولاته كريسبللو 34ج,كادبورى
4768,جهينه كوكتيل 1ل,جهينه
4769,دبل ديرى مشروب بنكهه ليمون 275مل,دبل ديرى
4770,Galaxy Smooth Milk Chocolate 24 x 36g,Galaxy


In [ ]:
df["brand_name"] = df["brand_name"].astype(str)

In [ ]:
tdf = DataFrame(df)

In [ ]:
tdf = tdf.assign(product_id=tdf.index_label)
tdf = tdf.drop(columns=["index_label"])
tdf

product_name,brand_name,product_id
صلصه هاينز برطمان خصم عرض,هاينز,2
بودرة عصير أناناس من سورس، 900 جم,سورس,4
بسكو مصر لوكس 6 قطعه علبه 12,بسكو مصر لوكس,5
Anise - 100g,nan,6
مسحوق برسيل جيل باللافندر - 3.9 كجم,برسيل,8
شكولاته الشمعدان بيور بندق,الشمعدان,9
لوبيا بلدى - 500 جم,nan,7
Dasani water 330ML,Dasani,3
ليمون اداليا 500 جم,nan,1
Americana Okra zero 400 gm,Americana,0


In [ ]:
tdf = tdf.select(["product_id", "product_name", "brand_name"])
tdf

product_id,product_name,brand_name
2,صلصه هاينز برطمان خصم عرض,هاينز
4,بودرة عصير أناناس من سورس، 900 جم,سورس
5,بسكو مصر لوكس 6 قطعه علبه 12,بسكو مصر لوكس
6,Anise - 100g,nan
8,مسحوق برسيل جيل باللافندر - 3.9 كجم,برسيل
9,شكولاته الشمعدان بيور بندق,الشمعدان
7,لوبيا بلدى - 500 جم,nan
3,Dasani water 330ML,Dasani
1,ليمون اداليا 500 جم,nan
0,Americana Okra zero 400 gm,Americana


In [ ]:
copy_to_sql(tdf, "products", "amurd", if_exists="replace")

## Load table from database

In [3]:
tdf = td_db.execute_query("SELECT * FROM amurd.products")
tdf = DataFrame(tdf)
tdf

product_id,product_name,brand_name,index_label
2,صلصه هاينز برطمان خصم عرض,هاينز,2
4,بودرة عصير أناناس من سورس، 900 جم,سورس,4
5,بسكو مصر لوكس 6 قطعه علبه 12,بسكو مصر لوكس,5
6,Anise - 100g,nan,6
8,مسحوق برسيل جيل باللافندر - 3.9 كجم,برسيل,8
9,شكولاته الشمعدان بيور بندق,الشمعدان,9
7,لوبيا بلدى - 500 جم,nan,7
3,Dasani water 330ML,Dasani,3
1,ليمون اداليا 500 جم,nan,1
0,Americana Okra zero 400 gm,Americana,0


In [4]:
# set_auth_token(base_url="test-environment-zxmlsctr5gru8885.env.clearscape.teradata.com")
tdf.apply(clean_text, env_name="iteration6")


RuntimeError: [Teradata][teradataml](TDML_2551) Authentication Token is required to run 'list_user_envs'. Set it using set_auth_token().

In [4]:
configure.ues_url

## Disconnect

In [ ]:
td_db.disconnect()